# Get data and label data-frame from excel file

Save a tuble of 'data' data-frame and 'label' series into jupyter space.

#### Input
1. %store -r excel_file_of_a_month 

#### Output 
1. %store data_label 


### Usage on the other notebook
    %store -r data_label
    data_label[0]  # features pandas data-frame
    data_label[1]  # labels pandas series
    
### Examine the data

    [x] 把 data frame 倒成 csv 的方法，用來檢查資料
        df.to_csv(file_name, sep='\t', encoding='utf-8')
    [x] 應用 https://pair-code.github.io/facets/ 可看出很多問題。  

    

In [1]:
import pandas as pd
import numpy as np
import peforth

### 定義 excel 表頭

    [ ] 與這裡的表頭比較，庫房的每月報表之「總表」裡偶有多出來的欄位，手動先剔除之。

In [2]:
%%js
alert("excel_to_data_label.ipynb: 與 'columns' list 比較，庫房的每月報表之「總表」裡偶有多出來的欄位，手動先剔除之。")
// 「總表」一般 worksheet.name 為 'Detail', 其中的欄位要照下表中 Title 的順序，而且不多、不少。
// 其中 AI 有用到的是下表中 flag 為 1 者，其他的可以刪除。我是希望照庫房日常工作報表的樣子無須加工直接沿用，因此這樣配合。
// 這還不止，總表的 Title 只有一 row 而且是第一 row, 以上的都要刪除，總表的右邊、下邊之外多的都要刪乾淨。

<IPython.core.display.Javascript object>

In [3]:
# 一次定義好，避免以後分好幾次出現，到時候容易出錯、不一致、改漏掉。

          #  title       flag  type
          #  =========== ====  ==========
columns = [('Customer'     ,1, str       ),  #  0
           ('ProjectName'  ,1, str       ),  #  1
           ('PartNo'       ,1, str       ),  #  2
           ('PartName'     ,0, str       ),  #  3
           ('Barcode'      ,1, str       ),  #  4
           ('ActionPlan'   ,0, str       ),  #  5
           ('TargetDate'   ,0, str       ),  #  6
           ('PM_CFM'       ,0, str       ),  #  7
           ('MStatus'      ,0, str       ),  #  8
           ('RefNo'        ,0, str       ),  #  9
           ('StoreInClass' ,0, str       ),  # 10
           ('Borrower'     ,0, str       ),  # 11
           ('BorrowerID'   ,0, str       ),  # 12
           ('BorrowerDEPT' ,1, str       ),  # 13
           ('Dept'         ,0, str       ),  # 14
           ('QTY'          ,1, np.int32  ),  # 15
           ('Price'        ,0, np.float32),  # 16
           ('Days'         ,1, np.int32  ),  # 17
           ('Tag'          ,1, str       )]  # 18
names    = [i[0] for i in columns]          # excel 總表 worksheet column title
features = [i[0] for i in columns if i[1]]  # selected features from
dtypes = {} 
for i in columns: dtypes = {**dtypes,**{i[0]:i[2]}} 


## Get training data and label

#### 到下一步，出錯說是無法把 'QTY' casting 成 integer

進 %pdb 察看 locals() 發現 'QTY' 有個 values 比實際多出一些。把 excel 表上底部之下看不見的部分 Alt-d-e 殺掉一些 rows 之後就好了。

In [4]:
%store -r excel_file_of_a_month
excel_file_of_a_month

'c:\\Users\\hcche\\WiBox\\wh300\\3月份W10W庫庫存報表b.xlsx'

In [7]:
%pdb off
df = pd.read_excel(excel_file_of_a_month, 
                   sheetname="Detail",  # 大小寫無關
                   header=0,  # Warehouse 的表多少會變動，不用它的，由本程式自己指定 names。
                   names=names, dtype=dtypes)

Automatic pdb calling has been turned OFF


In [8]:
# Tag 欄空白處要改成 0， TensorFlow 看不懂空白啥意思。 
df['Tag'] = df['Tag'].apply(lambda x: 1 if "1" in str(x) else 0)

In [9]:
df1 = df.query('Days >= 300');

In [10]:
df.shape

(6083, 19)

In [11]:
df1.shape

(776, 19)

In [ ]:
%%js
alert("excel_to_data_label.ipynb: df 全部，df1 僅 over 300d 看要用哪個，直接改程式，就在本行下面。")

In [14]:
# 挑出要用到的 columns
data = df1[features]

In [15]:
# 庫房來的資料「總表」中有「未領用」的，則「部門代碼」就會是空白的都剔掉，否則會造成類似 "Unable to get element as bytes." 的錯誤。
data = data.dropna(how="any", axis=0)  

In [16]:
# 挑出 Tag 欄，成為 label 
label = data.pop('Tag')
label[:4]

0    0
1    0
2    0
3    0
Name: Tag, dtype: int64

In [17]:
data[:4]

,Customer,ProjectName,PartNo,Barcode,BorrowerDEPT,QTY,Days
0,Molly,QP200,81.18R55.108G,2014050900478,20QK10,1,1392
1,Molly,QP200,81.18R55.108G,2014050900485,20QK10,1,1392
2,Molly,QP200,81.18R55.108G,2014050900492,20QK10,1,1392
3,Molly,QP200,81.18R55.108G,2014050900508,20QK10,1,1392


In [ ]:
# 核對一下 shapes 
%f __main__ :> data.shape . cr
%f __main__ :> label.shape . cr

### 9. IPython Magic - %store: Pass variables between notebooks.

The %store command lets you pass variables between two different notebooks.

    data = 'this is the string I want to pass to different notebook'
    %store data
    del data # This has deleted the variable

    Stored 'data' (str)

Now, in a new notebook...

    %store -r data
    print(data)

    this is the string I want to pass to different notebook

In [ ]:
data_label = (data,label)
%store data_label